# Assignment 2 - INFOMPPM - K.Y. He 5396131

Welcome to the algorithm behind my Social-Movie Recommender System. My Social-Movie Recommender System called 'PLCS' (pronounced: Places, named after the song *Made - Heights - All the Places*) is a movie recommender system where one is not only able to watch a wide range of different movies, but it is also a social networking platform where one can find friends or strangers to follow and discover what movies they are watching. PLCS allows its users to watch movies, share it on pubicly on their profile, rate movies and allow users to like or comment on other users' watched/ rated movies. PLCS used the platform *'Instagram'* as precedent and allows users to have the control of (publicly) sharing watched and rated movies.

PLCS included multiple pages to allow the user to browse through recommended movies, movies watched by *followings* and more. The algorithm behind the recommended movies is divided by the following 5 parts:
1. Preparing the data
2. Interface: Home Page
3. Interface: Following Page
4. Interface: Movies Page
5. Interface: Search Page

The first part is only preparing the data sets. The four remaining parts have their own purposes and will be further explained. 

# 1 Preparing the data
The provided MovieLens data set are prepared, merged together and checked for any missing values, before starting with building the algorithm.


## 1.1 Prepare the data sets

In [2]:
# import libraries
import pandas as pd

# load the datasets
movies_df = pd.read_csv('/Users/KayiuH/movies.csv', low_memory=False)
links_df = pd.read_csv('/Users/KayiuH/links.csv', low_memory=False)
ratings_df = pd.read_csv('/Users/KayiuH/ratings.csv', low_memory=False)
tags_df = pd.read_csv('/Users/KayiuH/tags.csv', low_memory=False)

In [105]:
# quick preview
# movies_df.head(3)

In [106]:
# quick preview
# links_df.head(3)

In [107]:
# quick preview
# ratings_df.head(3)

In [108]:
# quick preview
# tags_df.head(3)

In [3]:
print(movies_df.shape)
print(links_df.shape)
print(ratings_df.shape)
print(tags_df.shape)

(9742, 3)
(9742, 3)
(100836, 4)
(3683, 4)


## 1.2 Merge data sets

In [4]:
# The links_df dataset is not in my interest, but it's good to always check before deciding not to use it
# For now, we merge all data in one dataframe

movies_merged = pd.merge(left=movies_df, right=ratings_df, left_on='movieId', right_on='movieId')
movies_merged = pd.merge(left=movies_merged, right=tags_df, left_on='movieId', right_on='movieId')

# Since we won't be using it in our RS, we delete the timestamp columns
del movies_merged['timestamp_x']
del movies_merged['timestamp_y']

movies_merged.head(5)

,movieId,title,genres,userId_x,rating,userId_y,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,336,pixar
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,474,pixar
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,567,fun
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,336,pixar
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,474,pixar


In [5]:
# Checking for any missing data
movies_merged.isnull().sum()

movieId     0
title       0
genres      0
userId_x    0
rating      0
userId_y    0
tag         0
dtype: int64

# 2 Interface: Home Page

The Home Page is the first page when Jane logs in on her account. It consists of the following sections:

1. Movies frequently watched together
2. Movies similar to your recently watched
3. Movies similar to your recently rated
4. Surprise me!

## 2.1 Movies frequently watched together
This section consists of movies that are frequently paired with eachother. Users who watched *Movie A* will most likely like to watch *Movie B* as well. 

In [186]:
# Recommendation based on movies frequently watched together
from itertools import permutations

def create_combinations(x):
    combinations = pd.DataFrame(list(permutations(x.values, 2)), columns=['movie_1', 'movie_2'])
    return combinations

In [189]:
# Rating2 by userId_x
ratings_user = movies_merged.groupby(['userId_x']).size().reset_index(name='counts')
ratings_user = ratings_user.set_index('userId_x')

# Rating by userId_y
ratings_user_y = movies_merged.groupby(['userId_y']).size().reset_index(name='counts')
ratings_user_y = ratings_user_y.set_index('userId_y')

# more than 2 movies rated
ratings_user = ratings_user[ratings_user['counts']>2]
ratings_user = ratings_user.index

# Use create_combinations function
combi_mov = movies_merged.groupby('userId_x')['title'].apply(create_combinations)
# combi_mov .head()

# Make movie combinations that are frequently watched together
combi_mov  = combi_mov .reset_index(drop=True)
combi = combi_mov .groupby(['movie_1', 'movie_2']).size()
combi = combi.to_frame(name='counts').reset_index()

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            601, 602, 603, 604, 605, 606, 607, 608, 609, 610],
           dtype='int64', name='userId_x', length=610)

In [250]:
# Test on Forrest Gump
combi[combi['movie_1'] == 'Forrest Gump (1994)'].sort_values('counts', ascending=False).head(15)

,movie_1,movie_2,counts
581039,Forrest Gump (1994),Pulp Fiction (1994),374670
580404,Forrest Gump (1994),Fight Club (1999),77760
581259,Forrest Gump (1994),Star Wars: Episode IV - A New Hope (1977),38376
579977,Forrest Gump (1994),2001: A Space Odyssey (1968),31734
580768,Forrest Gump (1994),Léon: The Professional (a.k.a. The Professiona...,31500
580368,Forrest Gump (1994),Eternal Sunshine of the Spotless Mind (2004),30906
580113,Forrest Gump (1994),"Big Lebowski, The (1998)",25056
580624,Forrest Gump (1994),Inception (2010),25038
580425,Forrest Gump (1994),Forrest Gump (1994),23688
580331,Forrest Gump (1994),Donnie Darko (2001),20880


In [197]:
# searching for a movie that might fit
movie = input('\n Recently watched a movie? Great! We can pair you with a matching movie: ')

# Movie pairs
movies_paired = combi[combi['movie_1'] == movie].sample(4)
movies_paired = movies_paired.reset_index(drop=True)

# Test
print('\n Users that watched {} als watched: \n'.format(search))
movies_paired


 Recently watched a movie? Great! We can pair you with a matching movie: Braveheart (1995)

 Users that watched Braveheart (1995) als watched: 



,movie_1,movie_2,counts
0,Braveheart (1995),Big Fish (2003),1400
1,Braveheart (1995),Life of Pi (2012),300
2,Braveheart (1995),North by Northwest (1959),840
3,Braveheart (1995),Ghosts of the Abyss (2003),10


## 2.2 Movies similar to your recently watched

Our persona, Jane, has recently watched Gladiator (2000). Based on the jaccard similarity, the RS shall find a recommendation that it most similar to that movie.

In [247]:
# PART 1
# Get rid of all the duplicates
recommendations = movies_merged.copy().drop_duplicates(subset=['title'])

recommendations['genres'] = recommendations['genres'].str.split('|').tolist() # Split detects the | and add it to a list
recommendations = recommendations.explode('genres') # explodes all items in the list to a new row, the values in the other columns stay the same

##############
# PART 2
# Make a cross table
recommendations_genres = pd.crosstab(recommendations['title'], recommendations['genres'])

##############
# PART 3
# import necessary stuff
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform

##############
# PART 4
# calculate the distances
jaccard_distances = pdist(recommendations_genres.values, metric='jaccard')
square_jaccard_distances = squareform(jaccard_distances)

jaccard_similarity_array = 1 - square_jaccard_distances
distance_df = pd.DataFrame(jaccard_similarity_array, index=recommendations_genres.index, columns=recommendations_genres.index)

##############
# PART 5
# Check if its working with random movie and 4 suggestions per movie
distance_df['Gladiator (2000)'].sort_values(ascending=False).head(8)

title
Man in the Iron Mask, The (1998)               1.00
Hero (Ying xiong) (2002)                       1.00
Seven Samurai (Shichinin no samurai) (1954)    1.00
Sanjuro (Tsubaki Sanjûrô) (1962)               1.00
Gladiator (2000)                               1.00
Karate Kid, Part II, The (1986)                1.00
Treasure of the Sierra Madre, The (1948)       0.75
Great Escape, The (1963)                       0.75
Name: Gladiator (2000), dtype: float64

In [254]:
distance_df['Avengers, The (2012)'].sort_values(ascending=False).head(8)

title
Avengers, The (2012)                          1.0
Tron: Legacy (2010)                           1.0
Transformers: Revenge of the Fallen (2009)    1.0
Avatar (2009)                                 1.0
Godzilla (2014)                               1.0
Spider-Man 2 (2004)                           1.0
Star Trek (2009)                              1.0
Man of Steel (2013)                           0.8
Name: Avengers, The (2012), dtype: float64

## 2.3 Movies similar to your recently rated

Here collaborative filtering is used to discover correlation in movie ratings. Based on the pairways correlation, the RS is able to find our persona, Jane, a movie which is highly correlated to movies that she recently rated. She has recently rated the movie Interstellar (2014), which the algorithm will find a similar movie.

In [212]:
# PART 1
# In order to find start, we make a dataframe first with the average ratings and 
# the number of ratings for each movie

# Average ratings
movie_ratings = pd.DataFrame(movies_merged.groupby('title')['rating'].mean())

# Number of ratings per movie
movie_ratings['number_of_ratings'] = pd.DataFrame(movies_merged.groupby('title')['rating'].count())

##############
# PART 2
# Now we create a new table with users, title and the ratings
table = movies_merged.pivot_table(index = 'userId_x',
                                    columns = 'title', values = 'rating')

# Sort ratings data by number of ratings
# movie_ratings.sort_values('number_of_ratings', ascending = False).head(5)

##############
# PART 3
# Let's now test it on the movie "Fight Club"
fight_ratings = table['Fight Club (1999)']

##############
# PART 4
# Finding correlations in ratings from Fight Club to other movies (ratings)
similar_fight = table.corrwith(fight_ratings) # pairwise correlation

# Correlation of Fight Club with other movies (ratings)
corr_fight = pd.DataFrame(similar_fight, columns = ['correlation'])
corr_fight.dropna(inplace = True)

#corr_fight.head()

##############
# PART 5
# Number of ratings is being taken into account
# movies that correlated with Pulp Fiction and showed correlating high rating with it
corr_fight = corr_fight.join(movie_ratings['number_of_ratings'])

corr_fight[corr_fight['number_of_ratings']>150].sort_values('correlation', ascending = False).head(8)

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,correlation,number_of_ratings
title,,
Fight Club (1999),1.000000,11772
The Lego Movie (2014),0.743619,217
"Wolf of Wall Street, The (2013)",0.718489,270
Django Unchained (2012),0.635760,852
Life Is Beautiful (La Vita è bella) (1997),0.629640,704
Interstellar (2014),0.621685,730
The Revenant (2015),0.613049,217
Old Boy (2003),0.606980,312


To let Jane have control over her recommendations, the RS provides her with an input to search for a movie based on her recently rated movies!

In [215]:
# Part 1
# Search option
search_rating = input ('\n Recently rated a movie? Great! We can recommend you with a movie based on the rating:  ')

##############
# Part 2
# Search for a specic movie
search_r_movie = table[search_rating]

##############
# Part 3
# Find pairwise correlation for that movie
pairwise_correlation = table.corrwith(search_r_movie)

##############
# Part 4
# create new dataframe with correlating movie
correlated_movies = pd.DataFrame(pairwise_correlation, columns = ['correlation'])

##############
# Part 5
# Use the number of ratings columns
correlated_movies = correlated_movies.join(movie_ratings['number_of_ratings'])

##############
# Part 6
# Provide recommendations based on the recently rated movie
correlated_movies[correlated_movies['number_of_ratings']>150].sort_values('correlation', ascending=False).head(5)


 Recently rated a movie? Great! We can recommend you with a movie based on the rating:  Interstellar (2014)


/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,correlation,number_of_ratings
title,,
Interstellar (2014),1.000000,730
"Godfather: Part III, The (1990)",0.924658,225
"Life Aquatic with Steve Zissou, The (2004)",0.857159,222
Misery (1990),0.825168,308
Avengers: Infinity War - Part I (2018),0.759257,195


## 2.4 "Surprise me!"
The "Surprise me!" option allows Jane to have the interface provide **random suggestions**. These suggestions are not based on any personal preferences or previously watched movies and are completely based on chance. To allow Jane to have *some* control on her provided suggestions, it is still possible to select a genre. If Jane doesn't feel like watching any of her recommendations on her feed, but she does feel like watching something in the category "Romance" for example, she can simply search for it!

In [218]:
# For a completely random recommendation, we can do the following:
srprs = movies_merged.sample(8)
srprs

,movieId,title,genres,userId_x,rating,userId_y,tag
137746,2071,And the Band Played On (1993),Drama,387,3.5,474,AIDs
121423,1225,Amadeus (1984),Drama,64,4.0,474,Salieri
151890,2959,Fight Club (1999),Action|Crime|Drama|Thriller,189,4.0,599,psychological
6762,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,512,5.0,474,heist
3686,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,590,3.0,424,time travel
200550,27831,Layer Cake (2004),Crime|Drama|Thriller,122,4.5,62,British gangster
99916,750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,282,4.0,477,dark comedy
103801,924,2001: A Space Odyssey (1968),Adventure|Drama|Sci-Fi,29,5.0,599,relaxing


In [220]:
# But the RS can give Jane the control to choose her random suggestions
# by allowing her to chooise the genre
srprs_search = input('From which genre can I randomly suggest you a movie? ')
srprs_me = movies_2[movies_2['genres']== srprs_search].sample(8)
srprs_me

From which genre can I randomly suggest you a movie? Thriller


,movieId,title,genres,userId_x,rating,userId_y,tag
95949,593,"Silence of the Lambs, The (1991)",Thriller,221,4.0,477,suspense
49058,296,Pulp Fiction (1994),Thriller,251,5.0,424,Tarantino
158853,2959,Fight Club (1999),Thriller,522,4.0,599,philosophy
65103,296,Pulp Fiction (1994),Thriller,420,4.0,599,nonlinear narrative
54792,296,Pulp Fiction (1994),Thriller,318,4.5,599,organized crime
2624,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Thriller,226,4.0,477,time travel
33197,296,Pulp Fiction (1994),Thriller,96,5.0,599,fun
221586,79132,Inception (2010),Thriller,418,4.0,537,suspense


# 3 Interface: Following Page

The 'Following Page' does not provide an algorithm based on the MovieLens dataset. On this page, Jane can find the *'social aspect'* of PLCS. 

Jane follows friends and others on the PLCS platform and here, she can find what her *followings* are watching and rating. This page is similar to the instagram feed where she can browse through the movies that her *followings* have watched/ rated. Users are able to like and/or comment on the movies and find inspiration to watch the same movies. 

However, she can also find other users here that are in a way connected. As can be seen in the interface, the RS recommends a list of users that are socially connected with the *followings* of Jane. This allows Jane to think of following these people as well, as they have either the same taste in movies or are perhaps socially connected with Jane.

As this social aspect was beyond the scope of this algorithm, it has been agreed with Professors S. Shayan and A. Smits-Cardona that this part does not have to be implemented and only explained in words.

# 4 Interface: Movies Page

The 'Movie Page' is not based on recommendations, but rather a page where Jane can find multiple movies categorized in specific genres. For this page I choose to use the jaccard similarity used previously to find movies similar in genre.

In [153]:
recommendations_genre = movies_2[['title', 'genres']].copy()
search_page.sort_values('title', ascending = True).head(5)

,title,genres,rating,tag
216149,(500) Days of Summer (2009),Drama,4.5,romance
216181,(500) Days of Summer (2009),Comedy,2.0,romance
216180,(500) Days of Summer (2009),Romance,2.0,quirky
216180,(500) Days of Summer (2009),Drama,2.0,quirky
216180,(500) Days of Summer (2009),Comedy,2.0,quirky


In [ ]:
# calculate the distances
jaccard_distances = pdist(recommendations_genres.values, metric='jaccard')
square_jaccard_distances = squareform(jaccard_distances)

jaccard_similarity_array = 1 - square_jaccard_distances
distance_df = pd.DataFrame(jaccard_similarity_array, index=recommendations_genres.index, columns=recommendations_genres.index)

In [164]:
# In the category Romance, Jane have watched "Pretty woman"
distance_df['Pretty Woman (1990)'].sort_values(ascending=False).head(8)

title
Clueless (1995)               1.0
Adam's Rib (1949)             1.0
Coming to America (1988)      1.0
Wedding Crashers (2005)       1.0
Wedding Singer, The (1998)    1.0
Grumpier Old Men (1995)       1.0
Annie Hall (1977)             1.0
Pretty Woman (1990)           1.0
Name: Pretty Woman (1990), dtype: float64

In [165]:
# In the category Crime, Jane have watched "The Godfather: Part II"
distance_df['Godfather: Part II, The (1974)'].sort_values(ascending=False).head(8)

title
Shattered Glass (2003)              1.0
Shawshank Redemption, The (1994)    1.0
American History X (1998)           1.0
Better Luck Tomorrow (2002)         1.0
Pierrot le fou (1965)               1.0
Godfather: Part II, The (1974)      1.0
Au Hasard Balthazar (1966)          1.0
River's Edge (1986)                 1.0
Name: Godfather: Part II, The (1974), dtype: float64

In [168]:
# In the category Action, Jane have watched "Taken 2"
distance_df['Taken 2 (2012)'].sort_values(ascending=False).head(8)

title
Elite Squad (Tropa de Elite) (2007)                               1.0
Fight Club (1999)                                                 1.0
Clear and Present Danger (1994)                                   1.0
Léon: The Professional (a.k.a. The Professional) (Léon) (1994)    1.0
Taken 2 (2012)                                                    1.0
History of Violence, A (2005)                                     1.0
Hard-Boiled (Lat sau san taam) (1992)                             1.0
Munich (2005)                                                     1.0
Name: Taken 2 (2012), dtype: float64

In [169]:
# In the category Drama, Jane have watched "Forrest Gump"
distance_df['Forrest Gump (1994)'].sort_values(ascending=False).head(8)

title
Life Is Beautiful (La Vita è bella) (1997)    1.00
Forrest Gump (1994)                           1.00
(500) Days of Summer (2009)                   0.75
Hiroshima Mon Amour (1959)                    0.75
High Fidelity (2000)                          0.75
Roman Holiday (1953)                          0.75
Harold and Maude (1971)                       0.75
Bridget Jones's Diary (2001)                  0.75
Name: Forrest Gump (1994), dtype: float64

# 5 Interface: Search Page

The 'Search Page' is a page where Jane has the option to search for a movie in terms of title, genre or tag! When Jane feels like watching a movie in the category 'Action', she simply chooses to click on 'Action' on the interface and the algorithm will give her a random sample of Action movies! This is also applies to the "tag" option. If Jane feels like watching movies with 'Disney' or 'Romance', she can search on for it through the tags of each movie.

In [8]:
# The search page
search_page = recommendations[['title', 'genres', 'rating', 'tag']].copy()
search_page.sort_values('title', ascending = True).head()

,title,genres,rating,tag
215999,(500) Days of Summer (2009),Romance,4.0,artistic
215999,(500) Days of Summer (2009),Drama,4.0,artistic
215999,(500) Days of Summer (2009),Comedy,4.0,artistic
164451,...And Justice for All (1979),Thriller,4.0,lawyers
164451,...And Justice for All (1979),Drama,4.0,lawyers


In [13]:
## searching for specific movie
searchinput = input('\n What movie name are you searching? ')
searchoutput = search_page[search_page['title'] == searchinput]
searchoutput.head()


 What movie name are you searching? (500) Days of Summer (2009)


,title,genres,rating,tag
215999,(500) Days of Summer (2009),Comedy,4.0,artistic
215999,(500) Days of Summer (2009),Drama,4.0,artistic
215999,(500) Days of Summer (2009),Romance,4.0,artistic


In [256]:
# Part 1
## Search for a specific genre
search_input = input('\n What genre are you searching? ')

##############
# Part 2
# Movies with the specific genre
search_genre_output = search_page[search_page['genres'] == search_input]
search_genre_output = search_genre_output.reset_index(drop=True)

##############
# Part 3
# Print random sample within the specific genre
print('\n Random movies in this specific genre: '.format(search_input))
search_genre_output.sort_values('genres', ascending=False).sample(16)


 What genre are you looking for? Action

 Random movies in this specific genre: 


,title,genres,rating,tag
54295,Serenity (2005),Action,4.0,predictable
52043,"Lord of the Rings: The Return of the King, The...",Action,4.5,scenic
46142,Gladiator (2000),Action,4.0,Russell Crowe
69263,John Wick: Chapter Two (2017),Action,4.0,Heroic Bloodshed
2683,Congo (1995),Action,1.0,Michael Crichton
3106,Star Wars: Episode IV - A New Hope (1977),Action,4.0,engrossing adventure
39841,Fight Club (1999),Action,3.0,surreal
17885,Terminator 2: Judgment Day (1991),Action,4.0,Scifi masterpiece
63275,Inception (2010),Action,5.0,sci-fi
41212,Fight Club (1999),Action,5.0,complicated


In [241]:
# Part 1
## Search for a specific tag
search_input_tag = input('\n What tag are you searching? ')

##############
# Part 2
# Movies with the specific tag
search_tag_movies = search_page[search_page['tag'] == search_input_tag]
search_tag_movies = search_tag_movies.reset_index(drop=True)

##############
# Part 3
# Print random sample within the specific tag
print('\n Random movies with this specific tag: '.format(search_input_tag))
search_tag_movies.sort_values('tag', ascending=False).sample(5)


 What tag are you looking for? Disney

 Random movies with this specific tag: 


,title,genres,rating,tag
7558,Finding Nemo (2003),Comedy,4.0,Disney
5559,Alice in Wonderland (1951),Children,3.0,Disney
5783,Fantasia (1940),Children,5.0,Disney
4935,Mary Poppins (1964),Comedy,3.0,Disney
4630,Cinderella (1950),Fantasy,3.0,Disney
